## LW out by element

The main objective of this script is to estimate the average LWout for each element of the PV field (panels, soil and shade) for the mid-day period (10:00 - 15:00), in order to make a more detailed analysis and to understand the behavior of the PV field regarding its energy budget.

In [9]:
import pandas as pd
import openpyxl
import numpy as np
import glob
from plotnine import *
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from scipy import signal
import os

## Functions

In [24]:
def pvalue_text(p):
    if(p <= 0.001): p_text = '<.001'
    if(p > 0.001): p_text = '<.01'
    if(p > 0.01): p_text = '<.05'
    if(p > 0.05): p_text = p.round(2).astype(str)
    return(p_text)

In [39]:
drone_df = pd.read_csv("temperature.csv")

# convertiog date_time to proper format
drone_df['date_time'] = pd.to_datetime(drone_df['date_time'], format='%Y-%m-%d %H:%M:%S', utc=True)
drone_df['Month'] = drone_df['date_time'].dt.month
drone_df.loc[drone_df['Month'] == 7, 'Season'] = 'Summer'
drone_df.loc[drone_df['Month'] == 10, 'Season'] = 'Autumn'

drone_df = drone_df.loc[drone_df['Season'] == 'Autumn'].copy()

# rename columns
drone_df.rename(columns={'LW_obj_art_mast': 'Lout'}, inplace=True)

# Keep only relevant columns
drone_df = drone_df[['flight','label','date_time','Season','Lout']]

display(drone_df)

,flight,label,date_time,Season,Lout
0,20181024_1200,panel,2018-10-24 12:00:00+00:00,Autumn,542.328134
1,20181024_1200,shadow,2018-10-24 12:00:00+00:00,Autumn,483.644043
2,20181024_1200,soil,2018-10-24 12:00:00+00:00,Autumn,525.749863
3,20181024_1200,sun,2018-10-24 12:00:00+00:00,Autumn,442.418172
4,20181024_1430,panel,2018-10-24 14:30:00+00:00,Autumn,498.601921
...,...,...,...,...,...
147,20181031_1400,sun,2018-10-31 14:00:00+00:00,Autumn,312.375958
148,20181031_1515,panel,2018-10-31 15:15:00+00:00,Autumn,512.938421
149,20181031_1515,shadow,2018-10-31 15:15:00+00:00,Autumn,477.787147
150,20181031_1515,soil,2018-10-31 15:15:00+00:00,Autumn,488.735815


In [40]:
out_df = drone_df.pivot(index='date_time', columns='label', values=['Lout'])
out_df.columns = list(map('_'.join, out_df.columns.values))
out_df = out_df.reset_index()

display(out_df)

,date_time,Lout_panel,Lout_shadow,Lout_soil,Lout_sun
0,2018-10-24 12:00:00+00:00,542.328134,483.644043,525.749863,442.418172
1,2018-10-24 14:30:00+00:00,498.601921,473.800428,495.896162,291.551453
2,2018-10-24 15:00:00+00:00,495.305733,491.965564,503.758476,240.737165
3,2018-10-24 18:00:00+00:00,474.148610,476.217248,481.307030,236.369870
4,2018-10-25 06:30:00+00:00,467.669519,233.325006,468.936341,266.473124
5,2018-10-25 07:30:00+00:00,472.691646,232.435151,471.068278,253.072204
6,2018-10-25 08:45:00+00:00,467.367870,266.344198,471.678548,285.624114
7,2018-10-25 10:00:00+00:00,476.454713,233.026814,472.055736,266.016795
8,2018-10-25 12:00:00+00:00,487.573978,337.805454,482.288971,287.908305
9,2018-10-25 13:30:00+00:00,485.739833,233.499367,481.034167,252.535193


In [50]:
# t-test

a = out_df['Lout_panel']
b = out_df['Lout_soil']
t = stats.ttest_ind(a, b, equal_var=False, nan_policy='omit') # Welch t-test for inequal variances
print('P =', pvalue_text(t[1]))

P = <.01


In [4]:
# define mid-day and year
out18_df = out_df.loc[(out_df['date_time'].dt.hour >= 10) & (out_df['date_time'].dt.hour < 15) & (out_df['date_time'].dt.year == 2018)].copy() 

display(out18_df)

,date_time,LWout_panel,LWout_shadow,LWout_soil,LWout_sun
0,2018-10-24 12:00:00+00:00,542.328134,483.644043,525.749863,442.418172
1,2018-10-24 14:30:00+00:00,498.601921,473.800428,495.896162,291.551453
7,2018-10-25 10:00:00+00:00,476.454713,233.026814,472.055736,266.016795
8,2018-10-25 12:00:00+00:00,487.573978,337.805454,482.288971,287.908305
9,2018-10-25 13:30:00+00:00,485.739833,233.499367,481.034167,252.535193
13,2018-10-26 11:00:00+00:00,555.065009,478.956679,517.940426,499.356482
14,2018-10-26 13:30:00+00:00,547.377903,477.150811,517.792546,257.569140
17,2018-10-27 12:00:00+00:00,569.991925,472.605984,523.451355,550.066812
18,2018-10-27 13:00:00+00:00,572.304347,474.594261,528.888093,552.049294
21,2018-10-28 10:30:00+00:00,559.790108,473.706306,516.905893,553.726691


In [5]:
out18_df.describe()

,LWout_panel,LWout_shadow,LWout_soil,LWout_sun
count,19.000000,19.000000,19.000000,19.000000
mean,542.710510,448.116655,513.040803,399.065619
std,32.693498,82.775968,18.447350,116.476342
min,476.454713,233.026814,472.055736,252.535193
25%,533.021580,473.753367,505.472270,301.963705
50%,550.956631,480.958303,517.940426,341.627843
75%,567.776292,484.480023,524.600609,524.653127
max,583.547179,494.380633,535.933957,553.726691


In [6]:
# define mid-day and year
out19_df = out_df.loc[(out_df['date_time'].dt.hour >= 10) & (out_df['date_time'].dt.hour < 15) & (out_df['date_time'].dt.year == 2019)].copy() 

display(out19_df)

,date_time,LWout_panel,LWout_shadow,LWout_soil,LWout_sun
38,2019-07-17 11:30:00+00:00,633.056821,568.385105,605.382514,NaN
39,2019-07-17 13:30:00+00:00,629.459647,571.037747,615.512224,NaN
41,2019-07-18 10:00:00+00:00,577.858124,533.900970,555.134370,NaN
42,2019-07-18 12:00:00+00:00,635.740538,553.961505,596.558212,NaN
43,2019-07-18 14:30:00+00:00,637.515983,561.512748,607.627116,NaN
45,2019-07-19 12:15:00+00:00,612.885259,540.018193,590.607367,NaN
46,2019-07-19 14:00:00+00:00,606.169976,539.119949,592.567797,NaN
48,2019-07-20 11:00:00+00:00,600.621710,531.199373,569.145161,NaN
49,2019-07-20 12:00:00+00:00,611.847133,537.113526,587.388886,NaN
50,2019-07-20 13:30:00+00:00,618.623331,543.206356,598.918805,NaN


In [7]:
out19_df.describe()

,LWout_panel,LWout_shadow,LWout_soil,LWout_sun
count,16.000000,16.000000,16.000000,0.0
mean,612.488719,544.832042,589.593399,NaN
std,16.043371,12.745573,15.281123,NaN
min,577.858124,530.261586,555.134370,NaN
25%,600.533733,536.310387,579.859515,NaN
50%,612.366196,541.612275,591.587582,NaN
75%,621.332410,548.747025,597.184659,NaN
max,637.515983,571.037747,615.512224,NaN
